In [1]:
import os
import sys
import numpy as np
import PIL
from PIL import Image
import torch
# Append custom paths to sys.path for importing custom modules
sys.path.append(os.path.dirname("/home/student/Desktop/31171109-donotdelete/xai-chan/utils"))
from utils.models import ResNet_Model
from utils.transform import resize_transform

# Initialize device
device = torch.device('cuda:0')

In [5]:
# Load model
version = 50
downstream_task_model = ResNet_Model(version=version).to(device)
weights_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/result/imagenet/_Fold_2_5_400X_BreakHis_FT_60_resnet50_imagenet_/_37_96.07250755287009_95.06917631917632_0.9614846110343933.pth"  # TODO: Provide the model path
downstream_task_model.load_state_dict(torch.load(weights_path, map_location=device))
downstream_task_model.eval()

/home/student/anaconda3/envs/xai-chan/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/student/anaconda3/envs/xai-chan/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet_Model(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [6]:
# Prepare input
# image_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/benign/SOB/adenosis/SOB_B_A_14-22549AB/100X/SOB_B_A-14-22549AB-100-001.png"  # TODO: Provide the image path
# image_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-10147/100X/SOB_M_MC-14-10147-100-001.png"
image_path ="/home/student/Desktop/31171109-donotdelete/xai-chan/explanation/val_10/SOB_B_F_14-21998EF/400X/SOB_B_F-14-21998EF-400-001.png"
input_rgb = Image.open(image_path)
width, height = input_rgb.size
# print(f"Width: {width}, Height: {height}")
input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)
print(input_tensor)
outputs = downstream_task_model(input_tensor)
target_category = (outputs > 0.2).int().item()
print(target_category)

tensor([[[[0.6824, 0.6471, 0.6000,  ..., 0.6980, 0.6980, 0.7020],
          [0.6941, 0.6510, 0.5961,  ..., 0.6784, 0.6941, 0.6902],
          [0.6941, 0.6510, 0.5882,  ..., 0.6588, 0.6863, 0.6824],
          ...,
          [0.6510, 0.6588, 0.6627,  ..., 0.6706, 0.6706, 0.6588],
          [0.6353, 0.6510, 0.6549,  ..., 0.6706, 0.6627, 0.6510],
          [0.6235, 0.6353, 0.6510,  ..., 0.6431, 0.6431, 0.6392]],

         [[0.6980, 0.6588, 0.6000,  ..., 0.7020, 0.7020, 0.7098],
          [0.7137, 0.6627, 0.5922,  ..., 0.6824, 0.6941, 0.6980],
          [0.7176, 0.6627, 0.5843,  ..., 0.6549, 0.6824, 0.6941],
          ...,
          [0.6196, 0.6196, 0.6235,  ..., 0.6549, 0.6549, 0.6510],
          [0.6039, 0.6118, 0.6157,  ..., 0.6549, 0.6510, 0.6431],
          [0.5922, 0.5961, 0.6118,  ..., 0.6275, 0.6353, 0.6392]],

         [[0.7373, 0.7176, 0.6902,  ..., 0.7176, 0.7176, 0.7255],
          [0.7412, 0.7176, 0.6784,  ..., 0.7059, 0.7176, 0.7137],
          [0.7412, 0.7137, 0.6706,  ..., 0

In [12]:
from zennit.image import imgify, imsave  # For creating visualizations
from zennit.torchvision import ResNetCanonizer  # For ResNet-specific canonization
from zennit.composites import EpsilonPlusFlat  # For the composite function in LRP
from zennit.attribution import Gradient  # For attributing using gradients
from IPython.display import display
# Make sure your model is in evaluation mode
downstream_task_model.eval()

# Use the ResNet-specific canonizer
canonizer = ResNetCanonizer()

# Create a composite, specifying the canonizers
composite = EpsilonPlusFlat(canonizers=[canonizer])

target = torch.tensor([[1.0]]).to(device)

# Create the attributor, specifying model and composite
with Gradient(model=downstream_task_model, composite=composite) as attributor:
    # Compute the model output and attribution
    output, attribution = attributor(input_tensor, target)

print(f'Prediction: {output.item()}')  # Adapted for a binary classification output

# Sum over the channels
relevance = attribution.sum(1).cpu()

# Create an image of the visualized attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')



# Convert the image to RGB mode and then display it
img.convert('RGB').show()

# Show the image
# directly save the visualized attribution
imsave('/home/student/Desktop/31171109-donotdelete/xai-chan/explanation/explanation/SOB_B_F-14-21998EF-400-001-heatmap.png', relevance, symmetric=True, cmap='coldnhot')

Prediction: 3.891564119840041e-06


In [12]:

import torch.nn as nn

import torch

from zennit.attribution import Gradient, SmoothGrad
from zennit.core import Stabilizer
from zennit.composites import EpsilonGammaBox, EpsilonPlusFlat
from zennit.composites import SpecialFirstLayerMapComposite, NameMapComposite

from zennit.rules import Epsilon, ZPlus, ZBox, Norm, Pass, Flat
from zennit.types import Convolution, Activation, AvgPool, Linear as AnyLinear
from zennit.types import BatchNorm, MaxPool
from zennit.torchvision import VGGCanonizer, ResNetCanonizer
# Given that you're working with images in [0, 1] range
low, high = torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]])

composite = SpecialFirstLayerMapComposite(
    layer_map=[
        (nn.ReLU, Pass()),  # if your model uses ReLU activations
        (nn.AvgPool2d, Norm()),  # Normalize relevance for any Average Pooling layers
        (nn.Conv2d, ZPlus()),  # Any convolutional layer
        (nn.Linear, Epsilon(epsilon=1e-6)),  # Linear layers
        (nn.BatchNorm2d, Pass()),  # Ignore BatchNorm layers
    ],
    first_map=[
        (AnyLinear, ZBox(low, high))
    ]
)
target = torch.tensor([[target_category]]).float().to(device)
# directly save the visualized attribution
imsave('attrib-1.png', relevance, symmetric=True, cmap='coldnhot')

In [ ]:
import os
from PIL import Image
from torchvision import transforms
import torch

from zennit.image import imgify, imsave  # For creating visualizations
from zennit.torchvision import ResNetCanonizer  # For ResNet-specific canonization
from zennit.composites import EpsilonPlusFlat  # For the composite function in LRP
from zennit.attribution import Gradient  # For attributing using gradients
from IPython.display import display



def compute_heatmap(img_tensor, model, target):
    canonizer = ResNetCanonizer()
    composite = EpsilonPlusFlat(canonizers=[canonizer])
    
    with Gradient(model=model, composite=composite) as attributor:
        output, attribution = attributor(img_tensor, target)
    
    # Sum over the channels
    relevance = attribution.sum(1).cpu()
    return relevance

# Function to generate heatmaps for all images in a directory
def generate_heatmaps(directory, model):
    model.eval()
    target = torch.tensor([[1.0]]).to(device)

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).convert("RGB")
            img_tensor = resize_transform(img).unsqueeze(0).to(device)  # Add batch dimension and move to device

            relevance = compute_heatmap(img_tensor, model, target)
            
            # Save the heatmap
            heatmap_filename = f"{filename.split('.')[0]}_gradient.png"  # Removing the original extension and appending the method name
            heatmap_path = os.path.join(directory, heatmap_filename)
            imsave(heatmap_path, relevance, symmetric=True, cmap='coldnhot')

# Execute the function
generate_heatmaps("/path/to/image/directory", downstream_task_model)
